In [44]:
import tenseal as ts
import utils
import pandas as pd

In [45]:
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2 ** 40
context.generate_galois_keys()


In [46]:
secret_context = context.serialize(save_secret_key=True)
utils.write_data("keys/secret_key.txt", secret_context)

In [47]:
context.make_context_public()
public_context = context.serialize()
utils.write_data("keys/public_key.txt", public_context)

# Cryptage des données sensibles :

In [48]:
context=ts.context_from(utils.read_data("keys/secret_key.txt"))
data = pd.DataFrame({
   "Age": [45, 34, 67, 52 , 29],
    "Pression": [120, 135, 140, 110, 130],
    "Cardiaque": [80, 76, 85, 90, 72],
    "Température": [36.5, 37.0, 40.2, 38.7, 37.5]
})

In [49]:
age_crypted = ts.ckks_vector(context, data.Age.tolist())
pression_crypted = ts.ckks_vector(context, data.Pression.tolist())
cardiaque_crypted = ts.ckks_vector(context, data.Cardiaque.tolist())
temp_crypted = ts.ckks_vector(context, data.Température.tolist())
age_crypted=ts.ckks_vector(context, data.Age.tolist())
utils.write_data("outputs_client/age_crypted.txt", age_crypted.serialize())
utils.write_data("outputs_client/pression_crypted.txt", pression_crypted.serialize())
utils.write_data("outputs_client/cardiaque_crypted.txt", cardiaque_crypted.serialize())
utils.write_data("outputs_client/temp_crypted.txt", temp_crypted.serialize())

In [ ]:
m_proto = utils.read_data("outputs_server/average_age_encrypted.txt")
m = ts.lazy_ckks_vector_from(m_proto)
context=ts.context_from(utils.read_data("keys/secret_key.txt"))

m.link_context(context)
m=round(m.decrypt()[0], 2)
print("Âge moyen (approximatif) :",m)


Âge moyen (approximatif) : 3.2005218977441313e+19


# Decryptage 